In [3]:
!pip install -qqq guardrails-ai langchain openai wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


## Using Guardrails

Reference:
https://shreyar.github.io/guardrails/integrations/langchain/#create-prompt-template

In [4]:
rail_spec = """
<rail version="0.1">

<output>
    <object name="patient_info">
        <string name="gender" description="Patient's gender" />
        <integer name="age" format="valid-range: 0 100" />
        <string name="symptoms" description="Symptoms that the patient is currently experiencing" />
    </object>
</output>

<prompt>

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

{{doctors_notes}}

@complete_json_suffix_v2
</prompt>
</rail>
"""

In [5]:
import os
os.environ["OPENAI_API_KEY"]="PLACE_YOUR_KEY"
os.environ["WANDB_API_KEY"]="PLACE_YOUR_KEY"
os.environ["LANGCHAIN_API_KEY"]="PLACE_YOUR_KEY"

In [7]:
from rich import print
import openai
from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [8]:
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=openai.ChatCompletion.create)

In [9]:
print(output_parser.guard.base_prompt)

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's 
information.

{doctors_notes}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <object name="patient_info">
        <string name="gender" description="Patient's gender"/>
        <integer name="age" format="valid-range: min=0 max=100"/>
        <string name="symptoms" description="Symptoms that the patient is currently experiencing"/>
    </object>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{{'foo': 'example one'}}`
- `<list name='bar'><string format='upper-case' /></list>` => `{{"bar": ['STRING ONE', 'STRING TWO', etc.]}}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{{'baz': {{'foo': 'Some String', 'index': 1}}}}`

In [10]:
prompt = PromptTemplate(
    template=output_parser.guard.base_prompt,
    input_variables=output_parser.guard.prompt.variable_names,
)

In [11]:
model = OpenAI(temperature=0)


doctors_notes = """
49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream
"""
output = model(prompt.format_prompt(doctors_notes=doctors_notes).to_string())

In [12]:
print(output_parser.parse(output))


{
    'patient_info': {
        'gender': 'Male',
        'age': 49,
        'symptoms': 'Chronic macular rash to face & hair, worse in beard, eyebrows & nares. Itchy, flaky, slightly 
scaly. Moderate response to OTC steroid cream'
    }
}

## Other Samples

In [13]:
# profanity check
!pip install -qqq alt-profanity-check

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 30.3 MB/s eta 0:00:00


In [16]:
from profanity_check import predict

In [30]:
# Profanity Check

rail_spec ="""
<rail version="0.1">

<script language='python'>
from profanity_check import predict
from guardrails.validators import Validator, EventDetail, register_validator


from typing import Dict, List


@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    global predict
    global EventDetail
    def validate(self, key, value, schema) -> Dict:
        text = value
        prediction = predict([value])
        if prediction[0] == 1:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} contains profanity language",
                "",
            )
        return schema
</script>

<output>
    <string
        name="translated_statement"
        description="Translate the given statement into english language"
        format="is-profanity-free"
        on-fail-is-profanity-free="fix"
    />
</output>


<prompt>

Translate the given statement into english language:

{{statement_to_be_translated}}

@complete_json_suffix
</prompt>


</rail>
"""


In [25]:
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=openai.ChatCompletion.create)
prompt = PromptTemplate(
    template=output_parser.guard.base_prompt,
    input_variables=output_parser.guard.prompt.variable_names,
)

In [26]:
model = OpenAI(temperature=0)


In [36]:
statement_to_be_translated = "quesadilla de pollo"
output = model(prompt.format_prompt(statement_to_be_translated=statement_to_be_translated).to_string())

In [37]:
print(output)
print(output_parser.parse(output))

{"translated_statement": "Chicken Quesadilla"}

{'translated_statement': 'Chicken Quesadilla'}

In [38]:
statement_to_be_translated = "убей себя"
output = model(prompt.format_prompt(statement_to_be_translated=statement_to_be_translated).to_string())
print(output)
print(output_parser.parse(output))

{"translated_statement": "Kill yourself"}

{'translated_statement': ''}